In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


# Load the data
demand_data = pd.read_csv('demand.csv')
prices_data = pd.read_csv('prices.csv')
households_data = pd.read_csv('households.csv')
temp_data = pd.read_csv('Temperaturer.csv', sep=';')

# Clean temperature data
temp_data.columns = ['City', 'Timestamp_UTC', 'Temperature', 'Temperature24', 'Temperature48', 'Temperature72']
temp_data['Timestamp_UTC'] = pd.to_datetime(temp_data['Timestamp_UTC'])
temp_data['Date'] = pd.to_datetime(temp_data['Timestamp_UTC'].dt.date)
temp_data['Hour'] = temp_data['Timestamp_UTC'].dt.hour

# Create mapping dictionary for cities to price areas
city_to_area = {
    'Oslo': 'NO1', 'Sarpsborg': 'NO1', 'Moss': 'NO1',
    'Stavanger': 'NO2', 'Bergen': 'NO5', 'Trondheim': 'NO3',
    'Tromso': 'NO4', 'Tromsø': 'NO4', 'Evenes': 'NO4',
    'Kristiansand': 'NO2'
}

# Add price area to temperature data
temp_data['Price_area'] = temp_data['City'].map(city_to_area)
temp_data = temp_data[['Date', 'Hour', 'Price_area', 'Temperature']]
temp_data['Temperature'] = temp_data['Temperature'].str.replace(',', '.').astype(float)

# Merge data
merged_data = pd.merge(demand_data, households_data[['ID', 'Price_area']], on='ID', how='left')
merged_data = pd.merge(merged_data, prices_data[['Price_area', 'Date', 'Hour', 'Price_NOK_kWh']],
                      on=['Price_area', 'Date', 'Hour'], how='left')
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
merged_data = pd.merge(merged_data, temp_data, on=['Price_area', 'Date', 'Hour'], how='left')

# Remove duplicates
merged_data = merged_data.drop_duplicates()
print(f"Shape after removing duplicates: {merged_data.shape}")

# Create features
merged_data.loc[:, 'Day_of_week'] = merged_data['Date'].dt.dayofweek
merged_data.loc[:, 'Hour_of_day'] = merged_data['Hour']
merged_data.loc[:, 'Weekend'] = merged_data['Day_of_week'].isin([5, 6]).astype(int)
merged_data.loc[:, 'Month'] = merged_data['Date'].dt.month

# Remove rows with missing values
merged_data = merged_data.dropna(subset=['Price_NOK_kWh', 'Temperature', 'Demand_kWh']).copy()

# Prepare data for panel analysis
merged_data.loc[:, 'entity'] = merged_data['ID']
merged_data.loc[:, 'time'] = merged_data['Date'].astype(str) + '_' + merged_data['Hour'].astype(str)

# Set up panel data
panel_data = merged_data.set_index(['entity', 'time'])

# Prepare for random effects model
X_vars = ['Price_NOK_kWh', 'Temperature', 'Hour_of_day', 'Weekend']

# Select a subset of households if the dataset is too large
if len(merged_data['ID'].unique()) > 100:
    sample_households = np.random.choice(merged_data['ID'].unique(), 100, replace=False)
    print(f"Using a sample of {len(sample_households)} households for random effects model")
    panel_sample = panel_data[panel_data.index.get_level_values('entity').isin(sample_households)].copy()
else:
    panel_sample = panel_data.copy()

# Create dummy variables for hours and days
for hour in range(24):
    panel_sample.loc[:, f'hour_{hour}'] = (panel_sample['Hour_of_day'] == hour).astype(int)

for day in range(7):
    panel_sample.loc[:, f'day_{day}'] = (panel_sample['Day_of_week'] == day).astype(int)

# Drop one category to avoid perfect multicollinearity
X_vars.extend([f'hour_{h}' for h in range(1, 24)])
X_vars.extend([f'day_{d}' for d in range(1, 7)])

# Add month dummies for seasonality
for month in range(1, 13):
    panel_sample.loc[:, f'month_{month}'] = (panel_sample['Month'] == month).astype(int)

X_vars.extend([f'month_{m}' for m in range(2, 13)])

# Prepare formula for the random effects model
formula = 'Demand_kWh ~ ' + ' + '.join(X_vars)

print("\n=== Running Random Effects Model ===")
print(f"Sample size: {panel_sample.shape[0]} observations")
print(f"Number of households: {len(panel_sample.index.get_level_values('entity').unique())}")

try:
    # Fit random effects model
    model = RandomEffects.from_formula(formula, data=panel_sample)
    results = model.fit(cov_type='clustered', clusters=panel_sample.index.get_level_values('entity'))
    
    print("\n=== Random Effects Model Results ===")
    print(results.summary.tables[1])  # Print coefficients table
    
    # Extract and analyze price coefficient
    price_coef = results.params['Price_NOK_kWh']
    price_pvalue = results.pvalues['Price_NOK_kWh']
    
    print(f"\nPrice coefficient: {price_coef:.6f}")
    print(f"Price coefficient p-value: {price_pvalue:.6f}")
    
    # Calculate average price elasticity
    avg_price = panel_sample['Price_NOK_kWh'].mean()
    avg_demand = panel_sample['Demand_kWh'].mean()
    price_elasticity = (price_coef * avg_price) / avg_demand
    
    print(f"Average price elasticity: {price_elasticity:.6f}")
    
    # Calculate conditional elasticities (e.g., by time of day)
    print("\n=== Conditional Price Elasticities ===")
    
    # By weekend/weekday
    weekday_data = panel_sample[panel_sample['Weekend'] == 0].copy()
    weekend_data = panel_sample[panel_sample['Weekend'] == 1].copy()
    
    weekday_price = weekday_data['Price_NOK_kWh'].mean()
    weekday_demand = weekday_data['Demand_kWh'].mean()
    weekday_elasticity = (price_coef * weekday_price) / weekday_demand
    
    weekend_price = weekend_data['Price_NOK_kWh'].mean()
    weekend_demand = weekend_data['Demand_kWh'].mean()
    weekend_elasticity = (price_coef * weekend_price) / weekend_demand
    
    print(f"Weekday elasticity: {weekday_elasticity:.6f}")
    print(f"Weekend elasticity: {weekend_elasticity:.6f}")
    
    # By time of day (morning, day, evening, night)
    times = {
        'Morning (6-10)': (6, 10),
        'Day (10-17)': (10, 17),
        'Evening (17-23)': (17, 23),
        'Night (23-6)': (23, 6)
    }
    
    for time_label, (start, end) in times.items():
        if start < end:
            time_data = panel_sample[(panel_sample['Hour_of_day'] >= start) & (panel_sample['Hour_of_day'] < end)].copy()
        else:  # Handle night that wraps around midnight
            time_data = panel_sample[(panel_sample['Hour_of_day'] >= start) | (panel_sample['Hour_of_day'] < end)].copy()
        
        time_price = time_data['Price_NOK_kWh'].mean()
        time_demand = time_data['Demand_kWh'].mean()
        time_elasticity = (price_coef * time_price) / time_demand
        
        print(f"{time_label} elasticity: {time_elasticity:.6f}")
    
    # Calculate the Durbin-Watson statistic
    residuals = results.resids
    dw = sm.stats.stattools.durbin_watson(residuals)
    print(f"\nDurbin-Watson statistic: {dw:.4f}")
    
    # Check for model goodness of fit
    print(f"R-squared between: {results.rsquared_between:.4f}")
    print(f"R-squared within: {results.rsquared_within:.4f}")
    print(f"R-squared overall: {results.rsquared_overall:.4f}")

except Exception as e:
    print(f"Error running random effects model: {str(e)}")
    print("\nFalling back to simpler random effects implementation...")
    
    # Simplified approach using statsmodels directly
    from statsmodels.regression.mixed_linear_model import MixedLM
    
    # Prepare data for MixedLM
    model_data = panel_sample.reset_index().copy()
    
    # Simplified set of variables
    X_vars_simple = ['Price_NOK_kWh', 'Temperature', 'Weekend']
    X = model_data[X_vars_simple]
    X = sm.add_constant(X)
    y = model_data['Demand_kWh']
    groups = model_data['entity']
    
    try:
        # Fit mixed linear model with random intercepts
        mixed_model = MixedLM(y, X, groups=groups)
        mixed_results = mixed_model.fit()
        
        print("\n=== Simplified Random Effects Model Results ===")
        print(mixed_results.summary())
        
        # Extract price coefficient
        price_coef = mixed_results.params['Price_NOK_kWh']
        
        # Calculate average price elasticity
        avg_price = model_data['Price_NOK_kWh'].mean()
        avg_demand = model_data['Demand_kWh'].mean()
        price_elasticity = (price_coef * avg_price) / avg_demand
        
        print(f"Average price elasticity: {price_elasticity:.6f}")
        
    except Exception as e:
        print(f"Error running simplified random effects model: {str(e)}")

Shape after removing duplicates: (33871112, 7)
Using a sample of 100 households for random effects model

=== Running Random Effects Model ===
Sample size: 2942160 observations
Number of households: 100
Error running random effects model: name 'RandomEffects' is not defined

Falling back to simpler random effects implementation...

=== Simplified Random Effects Model Results ===
           Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Demand_kWh   
No. Observations: 2942160 Method:             REML         
No. Groups:       100     Scale:              0.9216       
Min. group size:  12581   Log-Likelihood:     -4055217.7089
Max. group size:  36639   Converged:          Yes          
Mean group size:  29421.6                                  
-----------------------------------------------------------
               Coef.  Std.Err.    z     P>|z| [0.025 0.975]
-----------------------------------------------------------
const           1.890    0.09